## Prepare Dependencies

In [1]:
import pandas as pd
import nltk
import spacy
from os import listdir
from os.path import isfile, join

from gensim import corpora, models

In [2]:
#Global Variables
nlp = spacy.load('en')

## Helper Functions

In [3]:
def string_maker(file_path):
    with open(str(file_path), 'r') as f_open:
        results = f_open.read()
    return results

## Wrangle Data

In [4]:
file_names = []
for file in listdir('saga/english/'):
    file_names.append(str(file))

print(file_names)

['laxdaela_saga.en2.txt', 'kormaks_saga.en.txt', 'haensna-thoris_saga.en2.txt', 'havardar_saga_isfirdings.en.txt', 'haensna-thoris_saga.en.txt', 'viga-glums_saga.en.txt', 'hrafnkels_saga_freysgoda.en.txt', 'viglundar_saga.en.txt', 'faereyinga_saga.en.txt', 'grettis_saga.en2.txt', 'laxdaela_saga.en.txt', 'egils_saga.en.txt', 'bandamanna_saga.en2.txt', 'heidarviga_saga.en.txt', 'grettis_saga.en.txt', 'eyrbyggja_saga.en.txt', 'thordar_saga_hredu.en.txt', 'eiriks_saga_rauda.en.txt', 'gunnlaugs_saga_ormstungu.en.txt', 'bandamanna_saga.en.txt', 'brennu-njals_saga.en.txt', 'gisla_saga_surssonar.en.txt']


In [5]:
#Import the text from every document as an entry in a dataframe
corpus = []
raw = pd.DataFrame()

for book in file_names:
    corpus.append(string_maker('saga/english/{}'.format(str(book))))

raw['saga_name'] = file_names
raw['text'] = corpus

In [6]:
raw.head()

,saga_name,text
0,laxdaela_saga.en2.txt,\nThe Story of the Laxdalers\n1903 translation...
1,kormaks_saga.en.txt,\nThe Saga of Cormac the Skald\n1901 translati...
2,haensna-thoris_saga.en2.txt,\nHænsa-Thori's Saga\n2002 translation into En...
3,havardar_saga_isfirdings.en.txt,\nThe Story of Howard the Halt\n1891 translati...
4,haensna-thoris_saga.en.txt,\nThe Story of Hen-Thorir\n1891 translation in...


## Clean Text

In [27]:
n = 0
chapters = []
docs = []
df = pd.DataFrame()

def book_frame(raw, num):
    
    chapter_title, para, chapter_text = [], [], []
    
    paragraphs, chapters = pd.DataFrame(), pd.DataFrame()
    
    for i, line in enumerate(raw.loc[0, 'text'].split('\n')):
        #print('{}: {}'.format(i, line))
    
        if len(line) > 0 and 'translation into English' not in line and not line.isspace():
            
            #Chapter + numeric
            if 'Chapter' in  line and any(char.isdigit() for char in line):
                chapter_text.append(line)
            else:
                docs.append(line)
    paragraphs['text'] = docs    
    return paragraphs

        
df = book_frame(raw, 0)
print(df)
#translation into English

                                                  text
0                           The Story of the Laxdalers
1    Ketil Flatneb hight a man, the son of Bjorn Ro...
2    In the latter days of Ketil came to pass the r...
3    After that had Ketil a very great guesting. Th...
4    Ketil Flatneb came in his ship to Scotland and...
5    Now Unn makes ready to depart from the Sheep-i...
6    After that gives Unn of her landtake to many m...
7    Olaf Feilan was the youngest of Thorstein's ch...
8    Herjolf and Thorgerd had been no long time tog...
9    At that time Hakon Aethelstan's fosterling was...
10   Hrapp hight a man; he dwelt in Laxriverdale, o...
11   Thord Goddi hight a man; he dwelt in Laxriverd...
12   Now it came to pass at the beginning of summer...
13   After that the king welcomed Hoskuld with mick...
14   Ingjald hight a man; he dwelt in Sheepisles; t...
15   Sithence stood up Ingjald and his folk and cla...
16   At that time cometh Asgaut home. Vigdis greete...
17   This 

In [8]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

## Topic Extraction